In [1]:
from functions import one_fell_reader, one_fell_dropper, one_fell_imputer, one_fell_encoder, one_fell_splitter, one_fell_modeler
import pandas as pd

Reading the csv files into pandas data frames

In [2]:
#file reading
file_list = ['application_train.csv','credit_card_balance.csv',
             'installments_payments.csv','POS_CASH_balance.csv','previous_application.csv']
app_train, ccb, ip, pcb, pa = one_fell_reader(file_list)

Prepping the data

In [3]:
#merging
ccb.drop('SK_ID_CURR',axis=1,inplace=True)
pcb = pcb.drop_duplicates('SK_ID_PREV')
pcb.drop('SK_ID_CURR',axis=1,inplace=True)
ip = ip.groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).agg('mean')
ip = ip.drop_duplicates('SK_ID_PREV')
pa = pa.drop_duplicates('SK_ID_CURR')
pa = pa.drop_duplicates('SK_ID_PREV')
ip_ccb = ip.merge(ccb, on='SK_ID_PREV', how='left')
ip_ccb_pcb = ip_ccb.merge(pcb,on='SK_ID_PREV',how='left')
ip_ccb_pcb.drop('SK_ID_CURR',axis=1, inplace=True)
pa_ip_ccb_pcb = pa.merge(ip_ccb_pcb, on='SK_ID_PREV', how='left')
pa_ip_ccb_pcb.drop('SK_ID_PREV',axis=1,inplace=True)
pa_ip_ccb_pcb = pa_ip_ccb_pcb.drop_duplicates('SK_ID_CURR')
super_df = app_train.merge(pa_ip_ccb_pcb, on='SK_ID_CURR', how='left') #final merge

In [4]:
#dropping all columns with 50% or more null values
super_df = one_fell_dropper(super_df)

In [5]:
#imputing missing data
super_df = one_fell_imputer(super_df)

In [6]:
#encoding all categorical data
super_df = one_fell_encoder(super_df,'TARGET','binary','encoders.pickle')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sk

In [7]:
#splitting the data for validation
x = super_df.drop('TARGET',axis=1)
y = super_df['TARGET']
x_train, x_test, y_train, y_test = one_fell_splitter(x,y,0.25,1)

creating the classification model

In [8]:
#creating the xgbclassifier model
model = one_fell_modeler(x_train,y_train,x_test,y_test)